In [ ]:
import pandas as pd

# Load Data

In [ ]:
from enfobench.datasets import ElectricityDemandDataset

ds = ElectricityDemandDataset("../data/electricity-demand")

In [ ]:
unique_ids = ds.metadata_subset.list_unique_ids()

In [ ]:
len(unique_ids)

In [ ]:
unique_id = unique_ids[0]
unique_id

In [ ]:
target, past_covariates, metadata = ds.get_data_by_unique_id(unique_id)

In [ ]:
target.info()

In [ ]:
past_covariates.info()

# Create Dataset

In [ ]:
from enfobench import Dataset

dataset = Dataset(
    target=target,
    past_covariates=past_covariates,
    metadata=metadata,
)

# Perform crossvalidation via server

In [ ]:
from enfobench.evaluation import ForecastClient

client = ForecastClient(port=3000)

In [ ]:
client.info()

In [ ]:
from enfobench.evaluation import cross_validate

crossval_df = cross_validate(
    client,
    dataset,
    start_date=pd.Timestamp("2023-01-01T00:00:00"),
    end_date=pd.Timestamp("2023-02-01T00:00:00"),
    horizon=pd.Timedelta("38 hours"),
    step=pd.Timedelta("1 day"),
)

In [ ]:
crossval_df

In [ ]:
cutoff_date_to_plot = crossval_df.cutoff_date.unique()[0]
crossval_df.loc[crossval_df.cutoff_date == cutoff_date_to_plot].set_index("timestamp").drop(
    columns=["cutoff_date"]
).plot()

# Evaluate metrics

In [ ]:
from enfobench.evaluation import evaluate_metrics
from enfobench.evaluation.metrics import mean_absolute_error, mean_bias_error

In [ ]:
evaluate_metrics(
    crossval_df,
    metrics={"MAE": mean_absolute_error, "MBE": mean_bias_error},
)

In [ ]:
metrics = evaluate_metrics(
    crossval_df,
    metrics={"MAE": mean_absolute_error, "MBE": mean_bias_error},
    groupby="cutoff_date",
)

In [ ]:
metrics

In [ ]:
metrics[["MAE", "MBE"]].plot()